# Nutrition Image Analysis using CNN

### Importing Neccessary Libraries

In [64]:
import numpy as np#used for numerical analysis
import tensorflow #open source used for both ML and DL for computation
from tensorflow.keras.models import Sequential #it is a plain stack of layers
from tensorflow.keras import layers #A layer consists of a tensor-in tensor-out computation function
#Dense layer is the regular deeply connected neural network layer
from tensorflow.keras.layers import Dense,Flatten
#Faltten-used fot flattening the input or change the dimension
from tensorflow.keras.layers import Conv2D,MaxPooling2D,Dropout #Convolutional layer
#MaxPooling2D-for downsampling the image
from tensorflow.keras.preprocessing.image import ImageDataGenerator


### Image Data Agumentation

In [65]:
#setting parameter for Image Data agumentation to the training data
train_datagen = ImageDataGenerator(rescale=1./255,shear_range=0.2,zoom_range=0.2,horizontal_flip=True)
#Image Data agumentation to the testing data
test_datagen=ImageDataGenerator(rescale=1./255)

In [66]:

import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.

if os.environ.get('RUNTIME_ENV_LOCATION_TYPE') == 'external':
    endpoint_38b0e8805e2a43f69f24eda44297d17a = 'https://s3.us.cloud-object-storage.appdomain.cloud'
else:
    endpoint_38b0e8805e2a43f69f24eda44297d17a = 'https://s3.private.us.cloud-object-storage.appdomain.cloud'

client_38b0e8805e2a43f69f24eda44297d17a = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='A4Ma569zz1m8LW6-LKaSOuUO69P-soQIRkAsdNe6xeQc',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url=endpoint_38b0e8805e2a43f69f24eda44297d17a)

streaming_body_1 = client_38b0e8805e2a43f69f24eda44297d17a.get_object(Bucket='image-donotdelete-pr-fgykpayifzhgei', Key='Dataset.zip')['Body']

# Your data file was loaded into a botocore.response.StreamingBody object.
# Please read the documentation of ibm_boto3 and pandas to learn more about the possibilities to load the data.
# ibm_boto3 documentation: https://ibm.github.io/ibm-cos-sdk-python/
# pandas documentation: http://pandas.pydata.org/


In [67]:
from io import BytesIO
import zipfile
unzip = zipfile.ZipFile(BytesIO(streaming_body_1.read()),'r')
file_paths = unzip.namelist()
for path in file_paths:
    unzip.extract(path)

In [68]:
pwd

'/home/wsuser/work'

In [69]:
import os
filenames = os.listdir('/home/wsuser/work/Dataset/TRAIN_SET')

### Loading our data and performing data agumentation

In [70]:
#performing data agumentation to train data
x_train = train_datagen.flow_from_directory('/home/wsuser/work/Dataset/TRAIN_SET',target_size=(64, 64),batch_size=5,color_mode='rgb',class_mode='sparse')
#performing data agumentation to test data
x_test = test_datagen.flow_from_directory('/home/wsuser/work/Dataset/TEST_SET',target_size=(64, 64),batch_size=5,color_mode='rgb',class_mode='sparse') 

Found 2626 images belonging to 5 classes.
Found 1055 images belonging to 5 classes.


In [71]:
print(x_train.class_indices)#checking the number of classes

{'APPLES': 0, 'BANANA': 1, 'ORANGE': 2, 'PINEAPPLE': 3, 'WATERMELON': 4}


In [72]:
from collections import Counter as c
c(x_train .labels)

Counter({0: 606, 1: 445, 2: 479, 3: 621, 4: 475})

In [73]:
print(x_train.class_indices)#checking the number of classes

{'APPLES': 0, 'BANANA': 1, 'ORANGE': 2, 'PINEAPPLE': 3, 'WATERMELON': 4}


In [74]:
from collections import Counter as c
c(x_train .labels)

Counter({0: 606, 1: 445, 2: 479, 3: 621, 4: 475})

### Creating the model

In [75]:
# Initializing the CNN
classifier = Sequential()

# First convolution layer and pooling
classifier.add(Conv2D(32, (3, 3), input_shape=(64, 64, 3), activation='relu'))
classifier.add(MaxPooling2D(pool_size=(2, 2)))

# Second convolution layer and pooling
classifier.add(Conv2D(32, (3, 3), activation='relu'))

# input_shape is going to be the pooled feature maps from the previous convolution layer
classifier.add(MaxPooling2D(pool_size=(2, 2)))

# Flattening the layers
classifier.add(Flatten())

# Adding a fully connected layer
classifier.add(Dense(units=128, activation='relu'))
classifier.add(Dense(units=5, activation='softmax')) # softmax for more than 2



In [76]:
classifier.summary()#summary of our model

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 62, 62, 32)        896       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 31, 31, 32)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 29, 29, 32)        9248      
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 6272)              0         
_________________________________________________________________
dense_4 (Dense)              (None, 128)               802944    
_________________________________________________________________
dense_5 (Dense)              (None, 5)                

### Compiling the model

In [77]:
# Compiling the CNN
# categorical_crossentropy for more than 2
classifier.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy']) 

## Fitting the model

In [78]:
classifier.fit_generator(
        generator=x_train,steps_per_epoch = len(x_train),
        epochs=20, validation_data=x_test,validation_steps = len(x_test))# No of images in test set

/opt/conda/envs/Python-3.8-main/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:1839: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/20
526/526 [==============================] - 30s 55ms/step - loss: 0.4785 - accuracy: 0.8198 - val_loss: 0.1042 - val_accuracy: 0.9431
Epoch 2/20
526/526 [==============================] - 29s 54ms/step - loss: 0.0339 - accuracy: 0.9907 - val_loss: 0.0965 - val_accuracy: 0.9716
Epoch 3/20
526/526 [==============================] - 29s 55ms/step - loss: 1.9646e-04 - accuracy: 1.0000 - val_loss: 0.0626 - val_accuracy: 0.9791
Epoch 4/20
526/526 [==============================] - 28s 54ms/step - loss: 1.0290e-04 - accuracy: 1.0000 - val_loss: 0.0878 - val_accuracy: 0.9782
Epoch 5/20
526/526 [==============================] - 29s 54ms/step - loss: 5.3924e-05 - accuracy: 1.0000 - val_loss: 0.1326 - val_accuracy: 0.9592
Epoch 6/20
526/526 [==============================] - 29s 55ms/step - loss: 2.0779e-05 - accuracy: 1.0000 - val_loss: 0.0922 - val_accuracy: 0.9791
Epoch 7/20
526/526 [==============================] - 28s 54ms/step - loss: 1.4865e-05 - accuracy: 1.0000 - val_loss: 0.

### Saving our model

In [79]:
# Save the model
classifier.save('nutrition.h5')

In [80]:
!tar -zcvf image-classification-mode_new.tgz nutrition.h5

nutrition.h5


In [81]:
ls -1

Dataset/
image-classification-mode_new.tgz
my_model.tar.gz
nutrition.h5


In [82]:
!pip install watson-machine-learning-client --upgrade

In [83]:
# Replace the credentials that you get from Watson Machine Learing service
from ibm_watson_machine_learning import APIClient
wml_credentials = {
                    'url': "https://us-south.ml.cloud.ibm.com",
                    'apikey':"j0a5QRH8YfliTikpZ3LNhWTMu_8N0teSw5F4agREb801"}
client = APIClient(wml_credentials)

In [84]:
client = APIClient(wml_credentials)

In [85]:
def guid_from_space_name(client,space_name):
    space = client.spaces.get_details()
    #print(space)
    return(next(item for item in space['resources'] if item['entity']["name"] == space_name)['metadata']['id'])

In [86]:
space_uid = guid_from_space_name(client,'image')
print("Space_UID = " + space_uid)

Space_UID = 4baa9931-047e-481e-adda-c61979e0a703


In [87]:
client.set.default_space(space_uid)

'SUCCESS'

In [88]:
client.software_specifications.list()

-----------------------------  ------------------------------------  ----
NAME                           ASSET_ID                              TYPE
default_py3.6                  0062b8c9-8b7d-44a0-a9b9-46c416adcbd9  base
pytorch-onnx_1.3-py3.7-edt     069ea134-3346-5748-b513-49120e15d288  base
scikit-learn_0.20-py3.6        09c5a1d0-9c1e-4473-a344-eb7b665ff687  base
spark-mllib_3.0-scala_2.12     09f4cff0-90a7-5899-b9ed-1ef348aebdee  base
ai-function_0.1-py3.6          0cdb0f1e-5376-4f4d-92dd-da3b69aa9bda  base
shiny-r3.6                     0e6e79df-875e-4f24-8ae9-62dcc2148306  base
tensorflow_2.4-py3.7-horovod   1092590a-307d-563d-9b62-4eb7d64b3f22  base
pytorch_1.1-py3.6              10ac12d6-6b30-4ccd-8392-3e922c096a92  base
tensorflow_1.15-py3.6-ddl      111e41b3-de2d-5422-a4d6-bf776828c4b7  base
scikit-learn_0.22-py3.6        154010fa-5b3b-4ac1-82af-4d5ee5abbc85  base
default_r3.6                   1b70aec3-ab34-4b87-8aa0-a4a3c8296a36  base
pytorch-onnx_1.3-py3.6         1bc6029

In [89]:
software_space_uid = client.software_specifications.get_uid_by_name("tensorflow_2.4-py3.7")
software_space_uid

'65e171d7-72d1-55d9-8ebb-f813d620c9bb'

In [90]:

model_details = client.repository.store_model(model='image-classification-mode_new.tgz',meta_props={
    client.repository.ModelMetaNames.NAME:"CNN",
    client.repository.ModelMetaNames.TYPE:"keras_2.2.4",
    client.repository.ModelMetaNames.SOFTWARE_SPEC_UID:software_space_uid}
                                             )
model_id = client.repository.get_model_uid(model_details)

Note: Warnings!! :  Model type keras_2.2.4 is deprecated. We recommend you use a supported model type. See Supported Frameworks https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/pm_service_supported_frameworks.html


In [91]:
model_id

'a28e609e-871e-40bc-b1ed-9b4a36d142f1'

In [93]:
client.repository.download(model_id,'my_model1.tar.gz')

Successfully saved model content to file: 'my_model1.tar.gz'


'/home/wsuser/work/my_model1.tar.gz'

In [94]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image

In [95]:
model = load_model("nutrition.h5")

In [96]:
y_pred = model.predict(x_test) 

In [97]:
import numpy as np
y_pred = np.argmax(y_pred, axis=1)#Returns the indices of the maximum values along an axis
y_pred

array([1, 3, 4, ..., 3, 1, 0])